In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/My Drive/LDS0_K287_Online_PhamMinhQuang/Project2/'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS0_K287_Online_PhamMinhQuang/Project2


In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=dae4eb886149e0b78f0f9b9cd9c26b4a71a862cdcd861e4d61214e5130551752
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('tiki_recommender').getOrCreate()

In [6]:
data = spark.read.options(header = 'True').csv("ReviewRaw.csv")

In [7]:
data.show(5,truncate=True)

+-----------+----------+-----------------+----------------+-------------------+------+------------------+--------------------+
|customer_id|product_id|             name|       full_name|       created_time|rating|             title|             content|
+-----------+----------+-----------------+----------------+-------------------+------+------------------+--------------------+
|     709310|  10001012| Lân Nguyễn Hoàng|Lân Nguyễn Hoàng|               null|     3|Ko dùng đc thẻ nhớ|Lúcđầu quên thông...|
|   10701688|  10001012| Nguyễn Khánh Hòa|Nguyễn Khánh Hòa|               null|     5|   Cực kì hài lòng|Tiki giao hàng nh...|
|   11763074|  10001012|  Toàn Phạm Khánh| Toàn Phạm Khánh|2019-04-17 15:42:45|     5|   Cực kì hài lòng|chất lượng camera...|
|    9909549|  10001012|Nguyen Quang Minh|            null|               null|     5|      Rất hài lòng|Hàng được đóng gó...|
|    1827148|  10001012|      Phạm Bá Đức|     Phạm Bá Đức|               null|     5|   Cực kì hài lòng|dễ cài

In [8]:
data_sub = data.select(['customer_id', 'product_id', 'rating'])

In [9]:
data_sub.count()

365821

In [10]:
from pyspark.sql.types import IntegerType
data_sub = data_sub.withColumn("customer_id", data_sub["customer_id"].cast(IntegerType()))
data_sub = data_sub.withColumn("product_id", data_sub["product_id"].cast(IntegerType()))
data_sub = data_sub.withColumn("rating", data_sub["rating"].cast(IntegerType()))

In [11]:
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import isnan, when, count, col

In [12]:
data_sub.show(5, truncate=True)

+-----------+----------+------+
|customer_id|product_id|rating|
+-----------+----------+------+
|     709310|  10001012|     3|
|   10701688|  10001012|     5|
|   11763074|  10001012|     5|
|    9909549|  10001012|     5|
|    1827148|  10001012|     5|
+-----------+----------+------+
only showing top 5 rows



In [13]:
data_sub.select([count(when(col(c).isNull(), c)).alias(c) for c in data_sub.columns]).toPandas().T

,0
customer_id,1721
product_id,1722
rating,1752


In [14]:
data_sub.select([count(when(isnan(c), c)).alias(c) for c in data_sub.columns]).toPandas().T

,0
customer_id,0
product_id,0
rating,0


In [15]:
#số lượng dòng null ít so với tổng số lượng dòng nên ta drop luôn
data_sub = data_sub.dropna(how="any", subset=["customer_id", "product_id", "rating"])

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [17]:
indexer1 = StringIndexer(inputCol='product_id', outputCol='product_id_idx')
indexer1_model = indexer1.fit(data_sub)
data_indexed = indexer1_model.transform(data_sub)
indexer2 = StringIndexer(inputCol='customer_id', outputCol='customer_id_idx')
indexer2_model = indexer2.fit(data_indexed)
data_indexed = indexer2_model.transform(data_indexed)

In [18]:
als = ALS(maxIter=5,
          regParam=1,
          rank = 25,
          userCol="customer_id_idx",
          itemCol="product_id_idx",
          ratingCol="rating",
          coldStartStrategy="drop",
          nonnegative=True)

In [19]:
(training, test) = data_indexed.randomSplit([0.8, 0.2])

In [20]:
fit_model = als.fit(training)
results = fit_model.transform(test)

In [21]:
results.select(["product_id_idx", "customer_id_idx", "rating", "prediction"]).show(5)

+--------------+---------------+------+----------+
|product_id_idx|customer_id_idx|rating|prediction|
+--------------+---------------+------+----------+
|         148.0|           22.0|     5| 3.5179036|
|         496.0|           40.0|     5| 3.9725387|
|         833.0|          360.0|     4| 3.3717666|
|        2866.0|          442.0|     5| 3.5855272|
|        1645.0|          582.0|     5| 3.6305914|
+--------------+---------------+------+----------+
only showing top 5 rows



In [22]:
evaluator = RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(results)
print("RMSE = " + str(rmse))

RMSE = 1.5842846192476427


độ chính xác tương đối ổn